# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
city_data = pd.read_csv("output_data/cities.csv")
city_data['City']=city_data['City'].str.title()
coordinates=city_data[['Lat', 'Lng']]
weight = city_data['Humidity']
max_intensity = city_data['Humidity'].max()
city_data

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Muros,42.7762,-9.0603,61.09,77,100,15.82,ES,1624133145
1,1,Hobart,-42.8794,147.3294,47.01,92,90,0.00,AU,1624132870
2,2,Ostrovnoy,68.0531,39.5131,48.52,80,39,8.03,RU,1624133145
3,3,Voh,-20.9667,164.7000,70.30,87,98,2.98,NC,1624133146
4,4,Isangel,-19.5500,169.2667,76.21,85,72,6.11,VU,1624133146
...,...,...,...,...,...,...,...,...,...,...
573,573,Cayenne,4.9333,-52.3333,86.04,75,75,1.01,GF,1624133529
574,574,Boli,45.7667,130.5167,53.04,97,12,4.34,CN,1624133529
575,575,Pangai,-19.8000,-174.3500,76.12,75,18,12.28,TO,1624133530
576,576,Buala,-8.1450,159.5921,79.83,80,75,4.88,SB,1624133530


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
figure_layout = {"width": "625px","height": "500px","border": "1px solid black","padding": "1px","margin": "0 auto 0 auto"}
fig = gmaps.figure(layout=figure_layout)

# gmaps.heatmap_layer(locations, weights=None, max_intensity=None, dissipating=True, point_radius=None, opacity=0.6, gradient=None)¶
fig.add_layer(gmaps.heatmap_layer(coordinates, weights=weight, max_intensity=max_intensity))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

#Store into variable named hotel_df
hotel_df = pd.DataFrame(city_data.loc[((city_data["Max Temp"]>=70) & (city_data["Max Temp"]<=80))& (city_data["Wind Speed"]<10) & (city_data["Cloudiness"] == 0)])

# Add a "Hotel Name" column to the DataFrame.
hotel_df['HotelName'] = ""
hotel_df

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
13,13,Alexandria,31.2156,29.9553,74.91,64,0,7.36,EG,1624132849,
172,172,Chapada Dos Guimaraes,-15.4606,-55.7497,74.86,28,0,6.42,BR,1624133198,
186,186,Marienburg,54.0359,19.0266,78.46,65,0,7.81,PL,1624133275,
212,212,Namangan,40.9983,71.6726,73.85,35,0,9.22,UZ,1624133283,
223,223,Gat,31.6100,34.7642,74.80,100,0,2.91,IL,1624133287,
253,253,Auce,56.4598,22.9017,72.05,84,0,6.22,LV,1624133297,
254,254,Alghero,40.5589,8.3181,77.09,78,0,9.22,IT,1624133297,
334,334,Vlore,40.4667,19.4897,70.72,79,0,4.59,AL,1624133321,
411,411,Aksu,41.1231,80.2644,73.42,28,0,5.44,CN,1624133415,
474,474,Tukrah,32.5341,20.5791,72.21,84,0,4.90,LY,1624133435,


In [30]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through hotel_df dataframe
# Set parameters to search each hotel within 5000 meters.
for index, row in hotel_df.iterrows():
    locations = str(row["Lat"])+","+str(row["Lng"])
    params = {
        "location": locations, 
        "radius": '5000',
        "type": "hotel",
        "key": g_key,
    }
    # Hit the Google Places API for each city's coordinates.
    response = requests.get(base_url, params=params).json()

    #Store the first Hotel result into the DataFrame.
    hotel_df.loc[index, 'HotelName'] = (response['results'][1]['name']).title()

In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt><b>Name</b></dt><dt>{HotelName}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [32]:
# Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info,  hover_text='Click for Hotel')

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…